In [8]:
import numpy
import io
from itertools import permutations

read training file and create 

In [9]:
def read_file_init_table(fname):
    tag_count = {}
    tag_count['<start>'] = 0
    word_tag = {}
    tag_trans = {}
    idx_line = 0
    is_first_word = 0
    
    with open(fname) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    content.insert(0, ("<kalimat id=1>"))
    i = 2
    for j in range(len(content)):
        if content[j] == '':
            content[j] = "</kalimat>"
            content.insert(j+1, "<kalimat id="+str(i)+">")
            i+= 1
    
    while idx_line < len(content):
        if not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                if content[idx_line] != '':
                    content_part = content[idx_line].split('\t')
                    if content_part[1] in tag_count:
                        tag_count[content_part[1]] += 1
                    else:
                        tag_count[content_part[1]] = 1

                    current_word_tag = content_part[0]+'|'+content_part[1]
                    if current_word_tag in word_tag:
                        word_tag[current_word_tag] += 1
                    else:    
                        word_tag[current_word_tag] = 1

                    if is_first_word == 1:
                        current_tag_trans = '<start>,'+content_part[1]
                        is_first_word = 0
                    else:
                        current_tag_trans = prev_tag+','+content_part[1]

                    if current_tag_trans in tag_trans:
                        tag_trans[current_tag_trans] += 1
                    else:
                        tag_trans[current_tag_trans] = 1                    
                    prev_tag = content_part[1]   
                
            else:
                tag_count['<start>'] += 1
                is_first_word = 1
            idx_line = idx_line + 1

        idx_line = idx_line+1       
    return tag_count, word_tag, tag_trans

In [2]:
def read_sample_file(fname):
    tag_count = {}
    tag_count['<start>'] = 0
    word_tag = {}
    tag_trans = {}
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    idx_line = 0
    is_first_word = 0
    is_last_word = 0

    while idx_line < len(content):
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                if content_part[1] in tag_count:
                    tag_count[content_part[1]] += 1
                else:
                    tag_count[content_part[1]] = 1

                current_word_tag = content_part[0]+'|'+content_part[1]
                if current_word_tag in word_tag:
                    word_tag[current_word_tag] += 1
                else:    
                    word_tag[current_word_tag] = 1

                if is_first_word == 1:
                    current_tag_trans = '<start>,'+content_part[1]
                    is_first_word = 0
                else:
                    current_tag_trans = prev_tag+','+content_part[1]

                if current_tag_trans in tag_trans:
                    tag_trans[current_tag_trans] += 1
                else:
                    tag_trans[current_tag_trans] = 1                    
                prev_tag = content_part[1]   

            else:
                tag_count['<start>'] += 1
                is_first_word = 1
            idx_line = idx_line + 1

        idx_line = idx_line+1       
    return tag_count, word_tag, tag_trans

In [7]:
# STATISTICAL METHOD
def read_dataset(fname):
    sentences = []
    tags = []
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content.insert(0, ("<kalimat id=1>"))
    i = 2
    for j in range(len(content)):
        if content[j] == '':
            content[j] = "</kalimat>"
            content.insert(j+1, "<kalimat id="+str(i)+">")
            i+= 1
    idx_line = 0
    while idx_line < len(content):
        sent = []
        tag = []
        if not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                if content[idx_line] != '':
                    content_part = content[idx_line].split('\t')
                    sent.append(content_part[0])
                    tag.append(content_part[1])
                
            idx_line = idx_line + 1
        sentences.append(sent)
        tags.append(tag)
        idx_line = idx_line+2        
    return sentences, tags

def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
    }


 
def transform_to_dataset(sentences, tags):
    X, y = [], []
 
    for sentence_idx in range(len(sentences)):
        for index in range(len(sentences[sentence_idx])):
            X.append(features(sentences[sentence_idx], index))
            y.append(tags[sentence_idx][index])
 
    return X, y
 

sentences,tags = read_dataset('Indonesian_Manually_Tagged_Corpus.txt')

# Split the dataset for training and testing
cutoff = int(.75 * len(sentences))
training_sentences = sentences[:cutoff]
test_sentences = sentences[cutoff:]
training_tags = tags[:cutoff]
test_tags = tags[cutoff:]
     
X, y = transform_to_dataset(training_sentences, training_tags)

from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
 
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', tree.DecisionTreeClassifier(criterion='entropy'))
])
clf.fit(X, y)   
 
print('Training completed')

X_test, y_test = transform_to_dataset(test_sentences, test_tags)
 
print("Accuracy:")
print(clf.score(X_test, y_test))

# Test model yang sudah dilatih dengan kalimat masukan bebas
from nltk import word_tokenize

def pos_tag(sentence):
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    #return zip(sentence, tags)
    return tags

# kalimat = ["Kera untuk amankan pesta olahraga .","Pemerintah kota Delhi mengerahkan monyet untuk mengusir monyet-monyet lain yang berbadan lebih kecil dari arena Pesta Olahraga Persemakmuran .", "Beberapa laporan menyebutkan setidaknya 10 monyet ditempatkan di luar arena lomba dan pertandingan di ibukota India .", "Pemkot Delhi memiliki 28 monyet dan berencana mendatangkan 10 monyet sejenis dari negara bagian Rajasthan .", "Jumlah monyet di ibukota India mencapai ribuan , sebagian besar , berada di kantor-kantor pemerintah dan hewan ini dianggap mengganggu ketertiban umum .", "Jenis monyet yang dikerahkan pemkot berbadan besar , berkekor panjang , dan memiliki wajah berwarna hitam .", "Monyet ini diikat dengan tali panjang dan pelatih yang mengawasi mereka akan melepas tali begitu monyet-monyet lain mendekat .", "Kantor berita AFP melaporkan stadion tinju dan hockey mendapatkan perhatian khusus karena sering diserbu monyet .", "Monyet besar akan dikerahkan di dua stadion untuk mengusir serbuan monyet kecil .", "Monyet besar akan dikerahkan di dua stadion tersebut untuk mengusir serbuan monyet kecil , kata Devender Prasad , pejabat pemerintah kota Delhi kepada kantor berita AFP ."]
sentence = "Monyet besar akan dikerahkan di dua stadion tersebut untuk mengusir serbuan monyet kecil , kata Devender Prasad , pejabat pemerintah kota Delhi kepada kantor berita AFP ."
print("Sentence : ", sentence)
print(pos_tag(word_tokenize(sentence)))

Training completed
Accuracy:
0.9384154767388301
Sentence :  Monyet besar akan dikerahkan di dua stadion tersebut untuk mengusir serbuan monyet kecil , kata Devender Prasad , pejabat pemerintah kota Delhi kepada kantor berita AFP .
['NN' 'JJ' 'MD' 'VB' 'IN' 'CD' 'NN' 'PR' 'SC' 'VB' 'NN' 'NN' 'JJ' 'Z' 'VB'
 'NNP' 'NNP' 'Z' 'NN' 'NN' 'NN' 'NNP' 'IN' 'NN' 'NN' 'NNP' 'Z']


In [10]:
tag_count, word_tag, tag_trans = read_file_init_table('Indonesian_Manually_Tagged_Corpus.txt')
print(tag_count)
print(word_tag)
print(tag_trans)

{'<start>': 9659, 'SC': 6200, 'NN': 30300, 'NNP': 16880, 'VB': 16009, 'RB': 2433, 'IN': 10196, 'Z': 12857, 'CC': 3512, 'CD': 8749, 'PR': 2941, 'JJ': 4900, 'MD': 2660, 'FW': 1185, 'PRP': 3607, 'NEG': 739, 'SYM': 1086, 'NND': 668, 'RP': 90, 'DT': 146, 'OD': 386, 'X': 190, 'WH': 126, 'UH': 12, 'fw': 1}
{'untuk|SC': 871, 'pesta olahraga|NN': 1, 'kota|NNP': 21, 'mengerahkan|VB': 1, 'monyet-monyet|NN': 2, 'yang|SC': 2809, 'lebih|RB': 353, 'dari|IN': 1189, 'Pesta Olahraga|NNP': 1, '.|Z': 4926, 'laporan|NN': 85, 'setidaknya|RB': 4, 'monyet|NN': 4, 'di|IN': 2095, 'arena|NN': 1, 'dan|CC': 2350, 'India|NNP': 49, 'Delhi|NNP': 4, '28|CD': 19, 'mendatangkan|VB': 4, 'Rajasthan|NNP': 1, 'ibukota|NNP': 1, 'mencapai|VB': 370, ',|Z': 7016, 'berada|VB': 125, 'kantor-kantor|NN': 4, 'ini|PR': 829, 'mengganggu|VB': 9, 'umum|JJ': 48, 'dikerahkan|VB': 2, 'berbadan|VB': 2, 'panjang|JJ': 11, 'wajah|NN': 3, 'hitam|JJ': 5, 'dengan|IN': 773, 'pelatih|NN': 1, 'mengawasi|VB': 9, 'akan|MD': 966, 'tali|NN': 1, 'lain|JJ

In [11]:
def create_trans_prob_table(tag_trans, tag_count):
    trans_prob = {}
    for tag1 in tag_count.keys():
        for tag2 in tag_count.keys():
            trans_idx = tag1+','+tag2
            if trans_idx in tag_trans:
                trans_prob[trans_idx] = tag_trans[trans_idx]/tag_count[tag1]
    return trans_prob

In [12]:
trans_prob = create_trans_prob_table(tag_trans, tag_count)

In [13]:
def create_emission_prob_table(word_tag, tag_count):
    emission_prob = {}
    for word_tag_entry in word_tag.keys():
        word_tag_split = word_tag_entry.split('|')
        if word_tag_split[0] != '' and word_tag_split[1] != '':
            current_word = word_tag_split[0]
            current_tag = word_tag_split[1]
            emission_key = current_word+','+current_tag
            emission_prob[emission_key] = word_tag[word_tag_entry]/tag_count[current_tag]    
    return emission_prob

In [14]:
emission_prob = create_emission_prob_table(word_tag, tag_count)

In [15]:
# BASELINE METHOD
def baselineMethod(sentence, word_tag):
    tag_sequence = []
    sentence_words = sentence.split()
    word_key = [key.split('|') for key in word_tag.keys()]
    # word_tag[]
    for i in sentence_words:
        val = 0
        curr_tag = ''
        for w in word_key:
            if i == w[0]:
                if val < word_tag[w[0]+'|'+w[1]]:
                    curr_tag = w[1]
                    val = word_tag[w[0]+'|'+w[1]]
        tag_sequence.append(curr_tag)
    return tag_sequence

tag_count, word_tag, tag_trans = read_file_init_table('Indonesian_Manually_Tagged_Corpus.txt')
tag_count_1, word_tag_1, tag_trans_1 = read_sample_file('sample_postagged.txt')
kalimat = ["Kera untuk amankan pesta olahraga .","Pemerintah kota Delhi mengerahkan monyet untuk mengusir monyet-monyet lain yang berbadan lebih kecil dari arena Pesta Olahraga Persemakmuran .", "Beberapa laporan menyebutkan setidaknya 10 monyet ditempatkan di luar arena lomba dan pertandingan di ibukota India .", "Pemkot Delhi memiliki 28 monyet dan berencana mendatangkan 10 monyet sejenis dari negara bagian Rajasthan .", "Jumlah monyet di ibukota India mencapai ribuan , sebagian besar , berada di kantor-kantor pemerintah dan hewan ini dianggap mengganggu ketertiban umum .", "Jenis monyet yang dikerahkan pemkot berbadan besar , berkekor panjang , dan memiliki wajah berwarna hitam .", "Monyet ini diikat dengan tali panjang dan pelatih yang mengawasi mereka akan melepas tali begitu monyet-monyet lain mendekat .", "Kantor berita AFP melaporkan stadion tinju dan hockey mendapatkan perhatian khusus karena sering diserbu monyet .", "Monyet besar akan dikerahkan di dua stadion untuk mengusir serbuan monyet kecil .", "Monyet besar akan dikerahkan di dua stadion tersebut untuk mengusir serbuan monyet kecil , kata Devender Prasad , pejabat pemerintah kota Delhi kepada kantor berita AFP ."]

accuracy = []
for i in range(len(kalimat)):
    print("\nKalimat ",i+1, " : ", kalimat[i])
    prediction = baselineMethod(kalimat[i], word_tag)
    print("\nPOS tag menggunakan baseline method : \n",prediction)
    
    real = baselineMethod(kalimat[i], word_tag_1)
    print("POS tag berdasarkan sampel: \n",real)

    true = 0
    for j in range(len(prediction)):
        if(prediction[j] == real[j]):
            true+=1
    accuracy.append(true/len(prediction)*100)
    print("Akurasi = %.2f" % accuracy[i])

print("\nRata-rata akurasi = %.2f" % (sum(accuracy)/len(kalimat)))


Kalimat  1  :  Kera untuk amankan pesta olahraga .

POS tag menggunakan baseline method : 
 ['', 'SC', 'VB', '', 'NN', 'Z']
POS tag berdasarkan sampel: 
 ['NN', 'SC', 'VB', '', '', 'Z']
Akurasi = 66.67

Kalimat  2  :  Pemerintah kota Delhi mengerahkan monyet untuk mengusir monyet-monyet lain yang berbadan lebih kecil dari arena Pesta Olahraga Persemakmuran .

POS tag menggunakan baseline method : 
 ['NN', 'NN', 'NNP', 'VB', 'NN', 'SC', 'VB', 'NN', 'JJ', 'SC', 'VB', 'RB', 'JJ', 'IN', 'NN', '', '', '', 'Z']
POS tag berdasarkan sampel: 
 ['NNP', 'NNP', 'NNP', 'VB', 'NN', 'SC', 'VB', 'NN', 'JJ', 'SC', 'VB', 'RB', 'JJ', 'IN', 'NN', '', '', 'NNP', 'Z']
Akurasi = 84.21

Kalimat  3  :  Beberapa laporan menyebutkan setidaknya 10 monyet ditempatkan di luar arena lomba dan pertandingan di ibukota India .

POS tag menggunakan baseline method : 
 ['CD', 'NN', 'VB', 'RB', 'CD', 'NN', 'VB', 'IN', 'NN', 'NN', '', 'CC', '', 'IN', 'NN', 'NNP', 'Z']
POS tag berdasarkan sampel: 
 ['CD', 'NN', 'VB', 'RB',

In [16]:
def viterbi(trans_prob, emission_prob, tag_count, sentence):
    #initialization
    viterbi_mat = {}
    tag_sequence = []
    path = []
    sentence_words = sentence.split()
    emission = [key.split(',') for key in emission_prob.keys()] # emission probability key split
    transmission = [key.split(',') for key in trans_prob.keys()] # transmission probability key split
    prev_tag = [['<start>',1]]
    best_tag = ''
    
    for i in range(0,len(sentence_words)):
        temp = []
        viterbi_mat[sentence_words[i]] = []
        for j in emission:
            if sentence_words[i] == j[0]:
                value = []
                for k in prev_tag:
                    for l in transmission:
                        if j[1] == l[1] :
                            key = sentence_words[i]+','+j[1]
                            if [k[0],j[1]] not in transmission:
                                val = 0
                            else :
                                val = k[1]*trans_prob[k[0]+','+j[1]]*emission_prob[key]
                            result = {'value' : val,'prev_tag':k[0],'tag' : j[1]}
                            if result not in viterbi_mat[sentence_words[i]]:
                                viterbi_mat[sentence_words[i]].append(result)
                            if [j[1],val] not in temp:
                                temp.append([j[1],val])
        prev_tag = copy.copy(temp)
    # GET BEST TAG
    i = len(sentence_words)-1
    max_val = 0
    max_tag = ''
    
    for j in viterbi_mat[sentence_words[i]]:
        if j['value'] > max_val:
            max_val = j['value']
            max_tag = j['tag']    
    tag_sequence.insert(0, max_tag)
    
    while i > 0 :
        max_val = 0
        max_tag = ''
        for j in viterbi_mat[sentence_words[i]]:
            if j['value'] > max_val:
                max_val = j['value']
                max_tag = j['prev_tag']
        tag_sequence.insert(0, max_tag)
        i-=1
    return viterbi_mat, tag_sequence

import copy 
tag_count, word_tag, tag_trans = read_file_init_table('Indonesian_Manually_Tagged_Corpus.txt')
tag_count_1, word_tag_1, tag_trans_1 = read_sample_file('sample_postagged.txt')
kalimat = ["Kera untuk amankan pesta olahraga .","Pemerintah kota Delhi mengerahkan monyet untuk mengusir monyet-monyet lain yang berbadan lebih kecil dari arena Pesta Olahraga Persemakmuran .", "Beberapa laporan menyebutkan setidaknya 10 monyet ditempatkan di luar arena lomba dan pertandingan di ibukota India .", "Pemkot Delhi memiliki 28 monyet dan berencana mendatangkan 10 monyet sejenis dari negara bagian Rajasthan .", "Jumlah monyet di ibukota India mencapai ribuan , sebagian besar , berada di kantor-kantor pemerintah dan hewan ini dianggap mengganggu ketertiban umum .", "Jenis monyet yang dikerahkan pemkot berbadan besar , berkekor panjang , dan memiliki wajah berwarna hitam .", "Monyet ini diikat dengan tali panjang dan pelatih yang mengawasi mereka akan melepas tali begitu monyet-monyet lain mendekat .", "Kantor berita AFP melaporkan stadion tinju dan hockey mendapatkan perhatian khusus karena sering diserbu monyet .", "Monyet besar akan dikerahkan di dua stadion untuk mengusir serbuan monyet kecil .", "Monyet besar akan dikerahkan di dua stadion tersebut untuk mengusir serbuan monyet kecil , kata Devender Prasad , pejabat pemerintah kota Delhi kepada kantor berita AFP ."]

trans_prob_1 = create_trans_prob_table(tag_trans_1, tag_count_1)
emission_prob_1 = create_emission_prob_table(word_tag_1, tag_count_1)

accuracy = []
for i in range(len(kalimat)):
    print("\nKalimat ",i+1, " : ", kalimat[i])
    matrix, tag_sequence = viterbi(trans_prob,emission_prob, tag_count,kalimat[i])
    print("\nPOS tag menggunakan baseline method : \n",tag_sequence)
    
    matrix_1, tag_sequence_1 = viterbi(trans_prob_1,emission_prob_1, tag_count_1,kalimat[i])
    print("POS tag berdasarkan sampel: \n",tag_sequence_1)

    true = 0
    for j in range(len(tag_sequence)):
        if(tag_sequence[j] == tag_sequence_1[j]):
            true+=1
    accuracy.append(true/len(tag_sequence)*100)
    print("Akurasi = %.2f" % accuracy[i])

print("\nRata-rata akurasi = %.2f" % (sum(accuracy)/len(kalimat)))


Kalimat  1  :  Kera untuk amankan pesta olahraga .

POS tag menggunakan baseline method : 
 ['', '', '', '', '', '']
POS tag berdasarkan sampel: 
 ['NN', 'SC', '', '', '', '']
Akurasi = 66.67

Kalimat  2  :  Pemerintah kota Delhi mengerahkan monyet untuk mengusir monyet-monyet lain yang berbadan lebih kecil dari arena Pesta Olahraga Persemakmuran .

POS tag menggunakan baseline method : 
 ['NNP', 'NNP', 'NNP', 'VB', 'NN', 'SC', 'VB', 'NN', 'JJ', 'SC', 'VB', 'RB', 'JJ', 'IN', '', '', '', '', '']
POS tag berdasarkan sampel: 
 ['NNP', 'NNP', 'NNP', 'VB', 'NN', 'SC', 'VB', 'NN', 'JJ', 'SC', 'VB', 'RB', 'JJ', 'IN', '', '', '', '', '']
Akurasi = 100.00

Kalimat  3  :  Beberapa laporan menyebutkan setidaknya 10 monyet ditempatkan di luar arena lomba dan pertandingan di ibukota India .

POS tag menggunakan baseline method : 
 ['CD', 'NN', 'VB', 'RB', 'CD', 'NN', '', 'IN', 'NN', '', '', '', '', '', '', '', '']
POS tag berdasarkan sampel: 
 ['CD', 'NN', 'VB', 'RB', 'CD', 'NN', 'NN', 'IN', 'NN',